# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:

import nltk
nltk.download(['stopwords','punkt', 'wordnet'])
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from prettytable import PrettyTable
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
import pickle
from sklearn.ensemble import AdaBoostClassifier
from keras.preprocessing.text import Tokenizer

[nltk_data] <urlopen error [Errno -2] Name or service not known>


In [32]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table("message_disaster", con=engine)
df.tail()


,id,message,original,genre,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
26211,30261,"The training demonstrated how to enhance micronutrient, protein and energy intake for meals using locally grown horticulture produce, rice and fish.",None,news,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26212,30262,A suitable candidate has been selected and OCHA Jakarta is currently working on her three month-contract starting on the 01st July 2005.,None,news,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26213,30263,"Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",None,news,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26214,30264,"Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.",None,news,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26215,30265,"A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.",None,news,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# Relevant data for the ML model are loaded.
# The data from the original message is skipped because we would need to know the language in order 
# to use a specific lemmatizer and process the text in the original language correctly. 
# The genre could be added in a second phase if we need to improve the model.



X = df['message']
y = df[df.columns[4:]]

In [33]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
y.tail()

/opt/anaconda/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
26211,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26215,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [38]:

def tokenize(text):
    
    """Normalizes and tokenizes a string in English (because of specific English lemmatizer) from a dataframe, 
    removes stopwords, lemmatizes. 
    
    """
    

    # normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    words = word_tokenize(text)
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok)
        clean_tokens.append(clean_tok)
    
    
    return clean_tokens

# test function tokenize

str_text = df.iloc[26214]['message'] # estracts the last row of the message column of the dataframe
tokenize(str_text)

['2',
 '000',
 'woman',
 'protesting',
 'conduct',
 'election',
 'teargassed',
 'tried',
 'converge',
 'local',
 'electoral',
 'commission',
 'office',
 'southern',
 'oil',
 'city',
 'port',
 'harcourt']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

The excercise from the UDACITY Lesson 4 (Machine Learning Pipelines) case study "Using a Pipeline" looks very similar to this project (is it a simpler text classification task), therefore is taken as a reference to build the first model. Only difference: the Random Forest Classifier is used as a parameter for the MultiOutputClassifier and not as stand-alone classifier. Other suitable classifiers for text classification are suggested here: https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f

In [39]:

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

  # train classifier
pipeline.fit(X_train.values, y_train.values)


/opt/anaconda/envs/python36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda/envs/python36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda/envs/python36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda/envs/python36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda/envs/pytho

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [41]:


pipeline.get_params()



{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7f78b5982950>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_feature

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [43]:
y_pred = pipeline.predict(X_test)


In the first attempt we display the metrics twice: once with the method classification_report (as recommende in the instructions)  and once with the method precision_recall_fscore_support as recommended here: https://stackoverflow.com/questions/48417867/access-to-numbers-in-classification-report-sklearn
We see that they deliver the same results (for each category) if the average of the precision_recall_fscore_support is set to None. If average="weighted" in precision_recall_fscore_support, then it returns the average/total values of the classification_report for each category --> we choose to use this metrics



In [44]:

def print_metrics(y_test, y_pred):
    
    """Displays the f1 score, precision and recall for each output category of the dataset comparing 2 methods.
         
    
    """

    for index, column in enumerate(y_test.columns):

        print("Column:", column)

        print(classification_report(y_test[column].values, y_pred.T[index]))

        precision,recall,fscore,support=score(y_test[column].values, y_pred.T[index],average="weighted")
        print ('Precision : '+"{:.2f}".format(precision))
        print ('Recall    : '+"{:.2f}".format(recall))
        print ('F-score   : '+"{:.2f}".format(fscore))
    
   
    return
  

  
print_metrics(y_test, y_pred)

    

Column: 0
              precision    recall  f1-score   support

           0       0.60      0.47      0.53      1971
           1       0.85      0.90      0.88      6627
           2       0.25      0.44      0.32        54

    accuracy                           0.80      8652
   macro avg       0.57      0.61      0.57      8652
weighted avg       0.79      0.80      0.79      8652

Precision : 0.79
Recall    : 0.80
F-score   : 0.79
Column: 1
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      7136
           1       0.81      0.42      0.55      1516

    accuracy                           0.88      8652
   macro avg       0.85      0.70      0.74      8652
weighted avg       0.87      0.88      0.87      8652

Precision : 0.87
Recall    : 0.88
F-score   : 0.87
Column: 2
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8624
           1       0.00      0.00      0.00        

/opt/anaconda/envs/python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8459
           1       0.80      0.04      0.08       193

    accuracy                           0.98      8652
   macro avg       0.89      0.52      0.53      8652
weighted avg       0.97      0.98      0.97      8652

Precision : 0.97
Recall    : 0.98
F-score   : 0.97
Column: 15
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8544
           1       0.29      0.02      0.03       108

    accuracy                           0.99      8652
   macro avg       0.64      0.51      0.51      8652
weighted avg       0.98      0.99      0.98      8652

Precision : 0.98
Recall    : 0.99
F-score   : 0.98
Column: 16
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      8355
           1       0.65      0.13      0.22       297

    accuracy                           0.97      8652
   m

In [45]:
def print_metrics_table(y_test, y_pred):
    
    """Displays the f1 score, precision and recall for each output category of the dataset as a pretty table
         
    
    """ 

    t = PrettyTable(["Column", "Precision", "Recall", "F1-Score"])
    
    for index, column in enumerate(y_test.columns):
        
        precision,recall,fscore,support=score(y_test[column].values, y_pred.T[index],average="weighted", warn_for=tuple())
        t.add_row([column, round(precision, 2), round(recall,2), round(fscore,2)])
    print(t)

In [46]:
print_metrics_table(y_test, y_pred)

+--------+-----------+--------+----------+
| Column | Precision | Recall | F1-Score |
+--------+-----------+--------+----------+
|   0    |    0.79   |  0.8   |   0.79   |
|   1    |    0.87   |  0.88  |   0.87   |
|   2    |    0.99   |  1.0   |   1.0    |
|   3    |    0.74   |  0.74  |   0.74   |
|   4    |    0.9    |  0.92  |   0.9    |
|   5    |    0.95   |  0.95  |   0.94   |
|   6    |    0.96   |  0.97  |   0.96   |
|   7    |    0.97   |  0.98  |   0.97   |
|   8    |    0.96   |  0.97  |   0.96   |
|   9    |    1.0    |  1.0   |   1.0    |
|   10   |    0.94   |  0.95  |   0.93   |
|   11   |    0.92   |  0.92  |   0.91   |
|   12   |    0.92   |  0.93  |   0.92   |
|   13   |    0.98   |  0.98  |   0.98   |
|   14   |    0.97   |  0.98  |   0.97   |
|   15   |    0.98   |  0.99  |   0.98   |
|   16   |    0.96   |  0.97  |   0.96   |
|   17   |    0.95   |  0.96  |   0.95   |
|   18   |    0.83   |  0.87  |   0.82   |
|   19   |    0.87   |  0.93  |   0.9    |
|   20   | 

### 6. Improve your model
Use grid search to find better parameters. 

https://towardsdatascience.com/optimizing-hyperparameters-in-random-forest-classification-ec7741f9d3f6 suggest to consider the following parameters to tune a random forest classifier:

n_estimators, 
max_depth, 
min_samples_split and
min_samples_leaf.

In [47]:
parameters =  { 
                  # "clf__estimator__n_estimators": [100, 300, 500], 
                  'clf__estimator__max_depth': [5, 30],
                  'clf__estimator__min_samples_split': [2, 5, 10],
                # 'clf__estimator__min_samples_leaf': [2, 3] 
    
              } 

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs = 4)

# no significant improvement using the gridsearch CV method, this will not be further investigated


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [48]:
# manually stopped because of long runtime

cv.fit(X_train, y_train)
y_pred2 = cv.predict(X_test)
print_metrics_table(y_test, y_pred2)


/opt/anaconda/envs/python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

In [ ]:
cv.get_params()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

The second best performing classifier from this project https://medium.com/@robert.salgado/multiclass-text-classification-from-start-to-finish-f616a8642538 will be tested as a second choice in the attempt to improve the model


In [49]:
#New the pipeline testing the second classifier
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=100, random_state=0)))
    ])



In [50]:
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7f78b5982950>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=100,
                                                      random_state=0),
    

In [51]:
pipeline2.fit(X_train, y_train)
y_pred3 = pipeline2.predict(X_test)


In [52]:
print_metrics_table(y_test, y_pred3)

+--------+-----------+--------+----------+
| Column | Precision | Recall | F1-Score |
+--------+-----------+--------+----------+
|   0    |    0.74   |  0.78  |   0.73   |
|   1    |    0.88   |  0.89  |   0.88   |
|   2    |    0.99   |  1.0   |   0.99   |
|   3    |    0.76   |  0.76  |   0.76   |
|   4    |    0.91   |  0.93  |   0.91   |
|   5    |    0.95   |  0.96  |   0.95   |
|   6    |    0.96   |  0.97  |   0.96   |
|   7    |    0.97   |  0.98  |   0.98   |
|   8    |    0.97   |  0.97  |   0.97   |
|   9    |    1.0    |  1.0   |   1.0    |
|   10   |    0.96   |  0.96  |   0.96   |
|   11   |    0.95   |  0.95  |   0.95   |
|   12   |    0.94   |  0.94  |   0.94   |
|   13   |    0.99   |  0.99  |   0.99   |
|   14   |    0.97   |  0.98  |   0.97   |
|   15   |    0.98   |  0.99  |   0.98   |
|   16   |    0.96   |  0.97  |   0.96   |
|   17   |    0.96   |  0.97  |   0.96   |
|   18   |    0.83   |  0.87  |   0.84   |
|   19   |    0.9    |  0.93  |   0.91   |
|   20   | 

In [ ]:
# improve adabost model with gridseach CV 
parameters =  { 
                   "clf__estimator__n_estimators": [100, 300, 500], 
                 # 'clf__estimator__max_depth': [5, 30],
                 # 'clf__estimator__min_samples_split': [2, 5, 10],
                # 'clf__estimator__min_samples_leaf': [2, 3] 
    
              } 

cv2 = GridSearchCV(pipeline2, param_grid=parameters, n_jobs = 4)

# no significant improvement using the gridsearch CV method, this will not be further investigated

In [ ]:
cv2.fit(X_train, y_train)
y_pred4 = cv2.predict(X_test)
print_metrics_table(y_test, y_pred4)

In [ ]:
cv2.get_params()

### 9. Export your model as a pickle file

In [ ]:

with open('model.pkl', 'wb') as f:
    pickle.dump(cv, f)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.